In [ ]:
wd<-"/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)
#library(hdf5r)
library(png)
#library(ggalluvial)
#library(tidyverse)
library(clustree)
library(RCTD)
library(Matrix)
#library(SPOTlight)
library(future)
library(grid)
library(RColorBrewer)
#library(pheatmap)
library(circlize)
library(igraph)
#library(ComplexHeatmap)
library(stringr)
#library(ComplexHeatmap)
library(cowplot)
library(biomaRt)
#library(scClustViz)
library(scales)
library(colorspace)
library(biomaRt)
library(clusterProfiler)
options(connectionObserver = NULL)
library(reshape2)

In [ ]:
da<-readRDS("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.merge.replace_v2.SCT.regress_CC.nC.mt.ident.pc20.k50.res02.rds")
da

### spatial cluster markers and GO terms

In [ ]:
Idents(da)<-da$SCT_snn_res.0.2
cluster_marker<-FindAllMarkers(da,logfc.threshold = 0.25,test.use = "wilcox",min.pct = 0.25 ,only.pos = T)

In [ ]:
write.csv(cluster_marker,"pc20_k50.res02.fc025.pct025.pos.clusterMarker.csv")

In [ ]:
cluster_marker<-read.csv("pc20_k50.res02.fc025.pct025.pos.clusterMarker.csv")
head(cluster_marker)

In [ ]:
cluster_marker %>%
    filter(p_val_adj<0.01) %>%
    group_by(cluster) %>% #filter(pct.1>0.75) %>%
    #top_n(n=15,wt=-p_val_adj) %>%
    top_n(n = 10, wt = avg_log2FC) ->top10

In [ ]:
options(repr.plot.width=8,repr.plot.height=10)
DoHeatmap(da, features = unique(top10$gene),group.colors = col,slot="scale.data"#,#remove.key=F,
         ) +
#+ NoLegend()
    #scale_fill_viridis()
    scale_fill_gradientn(colours = c('#4575B4','#91BFDB','#E0F3F8','white','#FC8D59','#D73027'))
ggsave("pc20.k50_res0.2_clusterMarker.pos.top10_fc.heatmap.RdYlBu.png",width = 8,height = 10,dpi = 400)

### cluster markers' functional pathway

In [ ]:
temp<- cluster_marker %>% filter(p_val_adj<0.01 )#avg_log2FC>0.3
    
table(temp$cluster)

In [ ]:
length(rownames(da))

backid<-bitr(rownames(da),fromType = "SYMBOL",toType = "ENTREZID",OrgDb = "org.Mm.eg.db")
dim(backid)

In [ ]:
backgenes<-backid[,2]
head(backgenes)
write.table(backgenes,"../WT.GOterm.SCT.backgeneids.txt",row.names = T,col.names = T)

In [ ]:
backgenes<-read.table("../WT.GOterm.SCT.backgeneids.txt")
backgenes<-as.character(backgenes$x)
head(backgenes)

In [ ]:
library(org.Mm.eg.db)
library(ggplot2)
#grlabs<-split(gene.nodle.df$gene,gene.nodle.df$module)
grlabs<-split(cluster_marker$gene,cluster_marker$cluster)
gcSample = lapply(grlabs, function(gr) as.numeric(bitr(gr, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Mm.eg.db")$ENTREZID))
pvalueCutoff = 0.01
qvalueCutoff = 0.01
xx.mus.go <- compareCluster(gcSample, OrgDb='org.Mm.eg.db', fun='enrichGO', 
                            pvalueCutoff = pvalueCutoff, qvalueCutoff = qvalueCutoff, ont = "BP", 
                            readable=T,universe=backgenes#,maxGSSize=100
                           )
#EnrichResGO_mm = xx.mus.go@compareClusterResult
#options(repr.plot.width=23,repr.plot.height=22)
#                  dotplot(xx.mus.go, title = paste0("Mouse Gene Ontology (qval < ", qvalueCutoff, ")"))
saveRDS(xx.mus.go,"pc20.k50.res02.cluster_fc025.pct025.pos.markergene_pqv001.go_220622.rds")

In [ ]:
xx.mus.go<-readRDS("pc20.k50.res02.cluster_fc025.pct025.pos.markergene_pqv001.go.rds")

In [ ]:
df<-xx.mus.go@compareClusterResult
head(df)
dim(df)
unique(df$Cluster)
library(dplyr)
df1 = df %>% group_by(.,Cluster) %>% filter(.,Count>=5) %>% top_n(.,-5,p.adjust)  #按照Cluster排序，并取每个cluster的前20个
df1$Description = factor(df1$Description,levels = rev(unique(df1$Description)))  #对Description进行按cluster排序，后面展示时同一cluster的go可以放在一起

In [ ]:
library(RColorBrewer)
options(repr.plot.width=10,repr.plot.height=8)
display.brewer.all()

In [ ]:
options(repr.plot.width=7.5, repr.plot.height=7)
library(ggplot2)
p=ggplot(df1,aes(Cluster,Description))+geom_point(aes(size=Count,color=p.adjust))+        #基因数目表示SIZE，p.adjust代表颜色深浅
    scale_color_gradientn(colors=brewer.pal(9,"Spectral"),guide = guide_colorbar(reverse = T))+
    theme_bw()+
    theme(axis.title = element_blank(),axis.text = element_text(size = 13))
p

In [ ]:
ggsave("WT.SCT.reg.pc20.k50.res02.cluster_fc025.pct025.pos.markergene_pqv001.count5.pval.top5.dotplot_220622.png",width = 7.5,height = 7,dpi = 400)

### time bulk DEGs according to dimain

In [ ]:
Idents(da)<-"time"
levels(Idents(da))

In [ ]:
wmda<-da[rownames(da),rownames(da@meta.data[da@meta.data$domain_res02=="WM",])]
unique(wmda$time)
unique(wmda$domain)
mgda<-da[rownames(da),rownames(da@meta.data[da@meta.data$domain_res02=="MG",])]
unique(mgda$time)
unique(mgda$domain)
vhda<-da[rownames(da),rownames(da@meta.data[da@meta.data$domain_res02=="VH",])]
unique(vhda$time)
unique(vhda$domain)
dhda<-da[rownames(da),rownames(da@meta.data[da@meta.data$domain_res02=="DH",])]
unique(dhda$time)
unique(dhda$domain)

In [ ]:
HT<-c(wmda,mgda,dhda,vhda)
HT

In [ ]:
name<-c("WM","MG","DH","VH")
name[1]

In [ ]:
sd<-"time_domain.marker/"
if(!dir.exists(sd))
    dir.create(sd)

for(i in c(1:4)){
    temp<-HT[[i]]
    mark<-FindAllMarkers(temp,logfc.threshold = 0.5,min.pct = 0.25,only.pos = F)
    mark$up_down<-NA
    mark[mark$avg_log2FC<0,"up_down"]<-"down"
    mark[mark$avg_log2FC>0,"up_down"]<-"up"
    mark$group<-paste(mark$cluster,name[i],mark$up_down,sep = "-")
    mark$group<-factor(mark$group,
                        levels = c(paste0('WT_sham-',name[i],'-up'),paste0('WT_3h-',name[i],'-up'),paste0('WT_24h-',name[i],'-up'),paste0('WT_72h-',name[i],'-up'),
                                   paste0('WT_sham-',name[i],'-down'),paste0('WT_3h-',name[i],'-down'),paste0('WT_24h-',name[i],'-down'),paste0('WT_72h-',name[i],'-down')    
                                                  ))
    head(mark)
    dim(mark)
    mark <- filter(mark,p_val_adj<0.01)
    dim(mark)
    table(mark$group)
    write.csv(mark,paste0(sd,"WT.SCT.",name[i],".time_Marker.p001.csv"))
    mark_up<-mark[mark$up_down=="up",]
    table(mark_up$group)
    unique(mark_up$group)
    mark_up$group<-factor(mark_up$group,levels = levels(mark$group)[1:4])
    table(mark_up$group)
    write.csv(mark_up,paste0(sd,"WT.SCT.",name[i],".time_upMarker.p001.csv"))
    mark_up %>%
        group_by(group) %>% #filter(pct.1>0.5) %>%
        top_n(n = 10, wt = avg_log2FC) -> top10
    options(repr.plot.width=12,repr.plot.height=12)
    DoHeatmap(temp, features = top10$gene) + NoLegend()
    ggsave(paste0(sd,name[i],".time_Marker.pos.pct025.top10_fc.heatmap.png"),width = 12,height = 12,dpi = 400)
    mark_up %>%
        group_by(group) %>% #filter(pct.1>0.5) %>%
        top_n(n = 30, wt = avg_log2FC) -> top30
    options(repr.plot.width=10,repr.plot.height=15)
    DoHeatmap(temp, features = top30$gene) + NoLegend()
    ggsave(paste0(sd,name[i],".time_Marker.pos.pct025.top30_fc.heatmap.png"),width = 10,height = 15,dpi = 400)
           
}

In [ ]:
for(j in c("WM","MG","VH","DH")){
    temp2<-HT[[j]]
    #name<-names(HT[j])
    for(a in c(levels(Idents(temp2))[2:4])){
        mark<-FindMarkers(temp2,logfc.threshold = 0.5,min.pct = 0.25,only.pos = F,ident.1=a,ident.2="WT_sham")
            
        mark <- filter(mark,p_val_adj<0.01)
        dim(mark)
        mark$group<-paste0(a,"_",j)
        table(mark$group)
        write.csv(mark,paste0(sd,"WT.SCT.",j,"_",a,".timevs.sham_Marker.np.p001.csv"))
    }
}

### merge domain deg and revise number barplot -220524

In [ ]:
sf<-"/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/time_domain.marker/"
f<-dir(sf,pattern = "WT.SCT.*.np.*csv")
f

In [ ]:
ht<-read.csv(paste0(sf,f[1]))
head(ht)

for(i in 2:length(f)){
    temp<-read.csv(paste0(sf,f[i]))
    ht<-rbind(ht,temp)
}
unique(ht$group)
write.csv(ht,"time_domain.marker/WT.domain.timevs.sham.fc05.pct025.p001.np.marker.csv")

In [ ]:
#domain bulk
ht$change<-"specific"
deg_3h_com<-Reduce(intersect,list(ht[ht$group=='WT_3h_WM',"X"],ht[ht$group=='WT_3h_MG',"X"],
                      ht[ht$group=='WT_3h_VH',"X"],ht[ht$group=='WT_3h_DH',"X"]))
length(deg_3h_com)
head(deg_3h_com)
ht[ht$group %in% c('WT_3h_WM','WT_3h_MG','WT_3h_VH','WT_3h_DH')& ht$X %in% deg_3h_com,"change"]<-"common"

deg_24h_com<-Reduce(intersect,list(ht[ht$group=='WT_24h_WM',"X"],ht[ht$group=='WT_24h_MG',"X"],
                      ht[ht$group=='WT_24h_VH',"X"],ht[ht$group=='WT_24h_DH',"X"]))
length(deg_24h_com)
head(deg_24h_com)
ht[ht$group %in% c('WT_24h_WM','WT_24h_MG','WT_24h_VH','WT_24h_DH')& ht$X %in% deg_24h_com,"change"]<-"common"

deg_72h_com<-Reduce(intersect,list(ht[ht$group=='WT_72h_WM',"X"],ht[ht$group=='WT_72h_MG',"X"],
                      ht[ht$group=='WT_72h_VH',"X"],ht[ht$group=='WT_72h_DH',"X"]))
length(deg_72h_com)
head(deg_72h_com)
ht[ht$group %in% c('WT_72h_WM','WT_72h_MG','WT_72h_VH','WT_72h_DH')& ht$X %in% deg_72h_com,"change"]<-"common"


In [ ]:
ht$up_down<-NA
ht$up_down<-ifelse(ht$avg_log2FC>0,"up","down")
unique(ht$up_down)

ht$time<-"sham"
ht[grepl("3h",ht$group,fixed = T) ,"time"]<-"3h"
ht[grepl("24h",ht$group,fixed = T) ,"time"]<-"24h"
ht[grepl("72h",ht$group,fixed = T) ,"time"]<-"72h"
ht$time<-factor(ht$time,levels = c("3h","24h","72h"))
head(ht)

In [ ]:
write.csv(ht,"time_domain.marker/WT.domain.timevs.sham.fc05.pct025.p001.np.marker_ad.csv")

In [ ]:
ht<-read.csv("time_domain.marker/WT.domain.timevs.sham.fc05.pct025.p001.np.marker_ad.csv")
head(ht)

### barplot to show the specific and common number of DEGs

In [4]:
temp<-ht

In [ ]:
temp$value<-ifelse(temp$up_down=="up",1,-1)
head(temp)
table(temp$up_down)

In [ ]:
temp2<-temp %>% group_by(group,time,up_down,change) %>% summarise(total_count=sum(value))
head(temp2)

temp2$HT<-"WM"
temp2[grepl("VH",temp2$group),"HT"]<-"VH"
temp2[grepl("DH",temp2$group),"HT"]<-"DH"
temp2[grepl("MG",temp2$group),"HT"]<-"MG"
unique(temp2$HT)
temp2$change<-factor(temp2$change,levels = c("specific","common"))
temp2$up_down<-factor(temp2$up_down,levels = c("up","down"))
unique(temp2$group)
temp2$group<-factor(temp2$group,levels = c('WT_3h_DH','WT_3h_MG','WT_3h_VH','WT_3h_WM',
                               'WT_24h_DH','WT_24h_MG','WT_24h_VH','WT_24h_WM',
                               'WT_72h_DH','WT_72h_MG','WT_72h_VH','WT_72h_WM'))
temp2$fill<-paste0(temp2$up_down,"_",temp2$change)
temp2$fill<-factor(temp2$fill,levels = c('up_specific','up_common','down_specific','down_common'))


In [ ]:
write.csv(temp2,"time_domain.marker/domain.time.vs.sham.barplot.number.csv")

In [ ]:
temp2<-read.csv("time_domain.marker/domain.time.vs.sham.barplot.number.csv")
head(temp2)

In [5]:
temp2$group<-factor(temp2$group,levels = c('WT_3h_WM','WT_3h_MG','WT_3h_DH','WT_3h_VH',
                               'WT_24h_WM','WT_24h_MG','WT_24h_DH','WT_24h_VH',
                                           'WT_72h_WM','WT_72h_MG',
                               'WT_72h_DH','WT_72h_VH'))

In [6]:
temp2$fill<-factor(temp2$fill,levels = c('down_specific','down_common','up_specific','up_common'))

In [ ]:
pdf("time_domain.marker/domain_time.vs.sham.np.deg.barplot_221029.pdf",width = 7,height = 6)
options(repr.plot.width=7,repr.plot.height=6)
ggplot(temp2,aes(x=group,y=total_count,fill=fill))+
    geom_bar(position = "stack",stat="identity",alpha=0.8)+
    scale_fill_manual(values = c('#2B83BA','#ABDDA4','#D7191C','#FDAE61'))+
    theme_bw()+
    theme(panel.grid  =element_blank(),axis.text.x.bottom = element_text(angle = 90),axis.line.x = element_line(colour = "black"))+
    coord_cartesian(ylim = c(-600,600))
#plot(p)
#abline(h = 0,col="black")
#ggsave("time_domain.marker/domain_time.vs.sham.np.deg.barplot_221029.png",width = 7,height = 6,dpi = 400)
dev.off()

In [ ]:
options(repr.plot.width=7,repr.plot.height=6)
ggplot(temp2,aes(x=group,y=total_count,fill=fill))+
    geom_bar(position = "stack",stat="identity")+
    scale_fill_manual(values = c('#2B83BA','#ABDDA4','#D7191C','#FDAE61'))+
    theme_bw()+
    theme(panel.grid  =element_blank(),axis.text.x.bottom = element_text(angle = 90),axis.line.x = element_line(colour = "black"))+
    coord_cartesian(ylim = c(-600,600))

### cluster composition each time-domain -220530

In [ ]:
df<-da@meta.data[,c("time","domain_res02","SCT_snn_res.0.2")]
head(df)
df$group<-paste0(df$time,"_",df$domain_res02)
df<-df[,c(1，3,4)]
colnames(df)<-c("time","cluster","group")
df$value<-1
head(df)
temp<-df %>% group_by(group,cluster) %>% summarise(spot_num=sum(value))
head(temp)
temp$group<-factor(temp$group,levels = c(
                                         #"g1",
                                         
                                        # "g2",
                                         'WT_sham_DH',
                                         'WT_3h_DH',
                                         'WT_24h_DH',
                                         'WT_72h_DH',
                                         #"g3",
                                         'WT_sham_VH',
                                        'WT_3h_VH',
                                        'WT_24h_VH',
                                        'WT_72h_VH',
                                         'WT_sham_MG',
                                         'WT_3h_MG',
                                         'WT_24h_MG',
                                        'WT_72h_MG',
                                        'WT_sham_WM',
                                         'WT_3h_WM',
                                         'WT_24h_WM',
                                         'WT_72h_WM'
                                        )
                   #c('WT_sham_WM','WT_sham_MG','WT_sham_DH','WT_sham_VH',
                   #                     'WT_3h_WM','WT_3h_MG','WT_3h_DH','WT_3h_VH',
                   #                     'WT_24h_WM','WT_24h_MG','WT_24h_DH','WT_24h_VH',
                   #                     'WT_72h_WM','WT_72h_MG','WT_72h_DH','WT_72h_VH')
                  )
options(repr.plot.width=7,repr.plot.height=6)
ggplot(temp,aes(x=group,y=spot_num,fill=cluster))+
    geom_bar(position = "stack",stat="identity")+
    scale_fill_manual(values = c('#66AB94','#5D3B83','#316B9B','#E2E293','#C580AA','#C13031','#E18727A8',"white"))+
    theme_bw()+
    theme(panel.grid  =element_blank(),axis.text.x.bottom = element_text(angle = 90),axis.line.x = element_line(colour = "black"))#+
    #coord_cartesian(ylim = c(0,100))
ggsave("/home/jovyan/zxli_SCI/result/Seurat/reg.CC/WT_replace_v2/res02_220310/domain_cluster.barplot.png",width =7,height = 6,dpi = 300)
